In [6]:

#import libraries
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models


In [7]:

# extract frames from mp4 files and return list of frames

total_frames = 1000#control how many frames to add


def getFrames(vidfile):
    vid = cv2.VideoCapture(vidfile)
    frames = []
    endframe = True
    t = 0
    while endframe:
        if t>total_frames:
            break
        endframe,frame = vid.read()
        #print(frame)
        if endframe:
            frames.append(frame)
            #print("processing frame: ",t)
            t+=1
    return frames

#read outpub vals for txtfile and make list    
def getSpeed(txtfile):
    speeds = []
    t=0
    with open(txtfile) as f:
        for line in f :
            if t>total_frames:
                break
            speed = line.rstrip('\n')
            speed = float(speed)
            speeds.append(speed)
            #print("speed at frame: ",t,"is: ",speed)
            t+=1
    return speeds





In [8]:
#get data from files into lists

path_txt = "data/train.txt"
path_mp4 = "data/train.mp4"


images_data= getFrames(path_mp4)
speed_data = getSpeed(path_txt)

In [11]:
images_data[0].shape,speed_data[0]
#confirm image and floats in lists


((480, 640, 3), 28.105569)

In [12]:

#generate optical flow output with raw data

def compute_dense_optical_flow(prev_image, current_image):

  #resize images 

  scale_percent = 20 # percent of original size
  width = int(prev_image.shape[1] * scale_percent / 100)
  height = int(prev_image.shape[0] * scale_percent / 100)
  dim = (width, height)
  
  # resize image
  prev_image = cv2.resize(prev_image, dim, interpolation = cv2.INTER_AREA)
  current_image = cv2.resize(current_image, dim, interpolation = cv2.INTER_AREA) 



  old_shape = current_image.shape
  prev_image_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
  current_image_gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

  assert current_image.shape == old_shape
  
  hsv = np.zeros_like(prev_image)
  hsv[..., 1] = 255
  flow = None
  flow = cv2.calcOpticalFlowFarneback(prev=prev_image_gray,
                                      next=current_image_gray, flow=flow,
                                      pyr_scale=0.8, levels=15, winsize=5,
                                      iterations=10, poly_n=5, poly_sigma=0,
                                      flags=10)

  mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
  hsv[..., 0] = ang * 180 / np.pi / 2
  hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

  return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) 


In [42]:
#Create optical flow images and list of mean speed
#features and labels we will be hitting the model with

opticalFlowFrames = []
speed_Data_final=[]
for i in range(0,len(images_data)-4):#trying to get better optical flow by comparing distant frames,                                                                 #instead of consecutive 

    image = compute_dense_optical_flow(images_data[i],images_data[i+4])
    opticalFlowFrames.append(image)
    mean_speed = (speed_data[i] + speed_data[i+1])
    label = np.asarray(mean_speed,dtype= np.float32)
    speed_Data_final.append(label)


In [43]:
#see optical flow result
img = Image.fromarray(opticalFlowFrames[4], 'RGB')
img.show()


In [17]:

#train and validation split

split_ratio = 0.8
train_data = opticalFlowFrames[:int(split_ratio*len(opticalFlowFrames))]
train_labels = speed_Data_final[:int(split_ratio*len(speed_Data_final))]

val_data = opticalFlowFrames[int(split_ratio*len(opticalFlowFrames)):]
val_labels = speed_Data_final[int(split_ratio*len(speed_Data_final)):]

In [21]:
len(train_labels),len(val_labels)


(797, 200)

In [22]:
#create tensorflow datasets using our features and labels 

train_dataset = tf.data.Dataset.from_tensor_slices((train_data,train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_data,val_labels))

#batch up our datasets 
train_dataset=train_dataset.batch(10)
val_dataset=val_dataset.batch(10)


In [23]:
model = models.Sequential()
#create our model


In [24]:
model.add(tf.keras.layers.Conv2D(24,(5,5), strides=2, padding='same',activation='relu',input_shape=(96, 128, 3)))

#convolutions
model.add(tf.keras.layers.Conv2D(36,(5,5), strides=2, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(48,(5,5), strides=2, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(64,(3,3), strides=1, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(64,(3,3), strides=1, padding='same',activation='relu'))

model.add(tf.keras.layers.Flatten())

#dense layers
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='relu'))

In [25]:
model.summary() #observe our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 64, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 32, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 16, 48)        43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 16, 64)        27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 16, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 12288)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               1

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MSE,
              metrics=['MSE'])


In [211]:
train_dataset.element_spec

(TensorSpec(shape=(None, 96, 128, 3), dtype=tf.int32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

In [27]:
# load weights if retraining
model.load_weights('./checkpoints/1')


In [28]:
model.fit(train_dataset, epochs=10,validation_data=val_dataset,use_multiprocessing=True,)


Train for 80 steps, validate for 20 steps
Epoch 1/10
80/80 [==============================] - 8s 95ms/step - loss: 4.4884 - MSE: 4.4922 - val_loss: 14.7613 - val_MSE: 14.7613
Epoch 2/10
80/80 [==============================] - 6s 70ms/step - loss: 1.4009 - MSE: 1.3824 - val_loss: 24.2879 - val_MSE: 24.2879
Epoch 3/10
80/80 [==============================] - 6s 72ms/step - loss: 1.5200 - MSE: 1.4974 - val_loss: 25.4948 - val_MSE: 25.4948
Epoch 4/10
80/80 [==============================] - 6s 71ms/step - loss: 1.2929 - MSE: 1.2784 - val_loss: 26.3009 - val_MSE: 26.3009
Epoch 5/10
80/80 [==============================] - 6s 72ms/step - loss: 1.2831 - MSE: 1.2748 - val_loss: 26.2670 - val_MSE: 26.2670
Epoch 6/10
80/80 [==============================] - 6s 75ms/step - loss: 1.0081 - MSE: 1.0011 - val_loss: 27.5537 - val_MSE: 27.5537
Epoch 7/10
80/80 [==============================] - 6s 76ms/step - loss: 0.7329 - MSE: 0.7337 - val_loss: 17.9852 - val_MSE: 17.9852
Epoch 8/10
80/80 [=========

In [31]:
model.save_weights('./checkpoints/3')

In [37]:
# Define the Keras TensorBoard callback.
from datetime import datetime 
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [38]:
model.fit(train_dataset, epochs=10,validation_data=val_dataset,use_multiprocessing=True,callbacks=[tensorboard_callback])


Train for 80 steps, validate for 20 steps
Epoch 1/10
80/80 [==============================] - 7s 82ms/step - loss: 1.9262 - MSE: 1.9289 - val_loss: 15.3139 - val_MSE: 15.3139
Epoch 2/10
80/80 [==============================] - 6s 79ms/step - loss: 0.7889 - MSE: 0.7886 - val_loss: 15.1308 - val_MSE: 15.1308
Epoch 3/10
80/80 [==============================] - 7s 91ms/step - loss: 2.2867 - MSE: 2.2833 - val_loss: 18.1100 - val_MSE: 18.1100
Epoch 4/10
80/80 [==============================] - 7s 87ms/step - loss: 1.2274 - MSE: 1.2295 - val_loss: 17.7400 - val_MSE: 17.7400
Epoch 5/10
80/80 [==============================] - 6s 76ms/step - loss: 1.6385 - MSE: 1.6432 - val_loss: 10.8958 - val_MSE: 10.8958
Epoch 6/10
80/80 [==============================] - 6s 77ms/step - loss: 1.5265 - MSE: 1.5261 - val_loss: 3.6207 - val_MSE: 3.6207
Epoch 7/10
80/80 [==============================] - 6s 80ms/step - loss: 2.6698 - MSE: 2.6781 - val_loss: 10.4097 - val_MSE: 10.4097
Epoch 8/10
80/80 [===========

In [ ]:
git filter-branch --index-filter 'git rm -r --cached --ignore-unmatch <data>' HEAD


In [39]:
!tensorboard --logdir logs

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.2.1 at http://localhost:6007/ (Press CTRL+C to quit)
^C
